In [ ]:
# Import necessary packages
import os
import pathlib
import pandas as pd # Aggregating, data manipulation
import re # Parsing information out of file names
import rioxarray as rxr # Work with raster data
import geopanda as gpd # Work with vector data
import 

In [ ]:
# Set up a project data directory

# Define your study area

* Download the USFS National Grassland Units
* Select your study sites

In [ ]:
# Download from the URL
# Define a path to save the study area
# Only download once
if not os.path.exists(study_area_path):
    # Download study area

    # Save the study area

# Load the study area

# Plot to check that the data is as expected

In [ ]:
# Select the study sites

# Make two site maps

In [ ]:
# Define the download URL for the study area


# Load in soil data

For each grassland:
  * Download model variable as raster layers covering your study area envelope, including at least one soil variable from the POLAris dataset.

In [ ]:
# Define download URL template for the soil data

# List out all soil files needed for download

output_da_list = [  ]

# Loop through each of the soil files (more info in redlining 92)

    # Download raster data

    # Extract metadata from the file name

    # Unzip the raster data

    # Only download once -- Save raster data to the disk

    # Load the raster data into Python, mask, and scale.

    # Make sure that the CRSs match between vector and raster data (Get the study bounds section)

    # Crop the raster data

output_da_list.append(cropped_da)

# merge tiles

# Make a plot to check teh download (don't need to make a cloud mask), load in multiple bands, or calcualte the spectral indices

In [ ]:
# Merge tile